import engine as eng
import argparse
from onnx import ModelProto
import tensorrt as trt 

engine_name = "mlsd.trt"
onnx_path = "mlsd_model.onnx"
batch_size = 1 

model = ModelProto()
with open(onnx_path, "rb") as f:
    model.ParseFromString(f.read())

d0 = model.graph.input[0].type.tensor_type.shape.dim[1].dim_value
d1 = model.graph.input[0].type.tensor_type.shape.dim[2].dim_value
d2 = model.graph.input[0].type.tensor_type.shape.dim[3].dim_value
shape = [batch_size , d0, d1 ,d2]
engine = eng.build_engine(onnx_path, shape= shape)
eng.save_engine(engine, engine_name) 

In [1]:
import numpy as np
from PIL import Image
import tensorrt as trt
import skimage.transform



HEIGHT = 512
WIDTH = 512
input_shape=[HEIGHT, WIDTH]

def rescale_image(image, output_shape, order=1):
    image = skimage.transform.resize(image, output_shape,
               order=order, preserve_range=True, mode='reflect')
    return image





In [7]:
import engine as eng
import inference as inf
import numpy as np
from PIL import Image
import tensorrt as trt
import skimage.transform
from log import timer, logger
import trt_common
import torchvision.transforms as T
def load_data(path):
    trans = T.Compose([
        T.Resize((512,512)), T.ToTensor()
    ])

    img = Image.open(path)
    img_tensor = trans(img).unsqueeze(0)
    return np.array(img_tensor)



engine_name = "mlsd.trt"
onnx_path = "mlsd_model.onnx"
batch_size = 1 
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)

load_trt = timer("Load TRT Engine")
engine = eng.load_engine(trt_runtime, engine_name)
load_trt.end()



-------------------------------------------------- 

Load TRT Engine... Done (0.30206s)



In [9]:
# allocate buffer
infer_trt = timer("TRT Inference")

inputs, outputs, bindings, stream = trt_common.allocate_buffers(engine)

# load data
inputs[0].host = load_data('/home/cheng/mlsd/test_photo.jpg')

# inference
with engine.create_execution_context() as context:
    trt_outputs = trt_common.do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
preds = trt_outputs[0]
infer_trt.end()

-------------------------------------------------- 

TRT Inference... Done (0.45467s)



In [17]:
trt_outputs

[array([0.02436978, 0.02356762, 0.02284408, 0.02232116, 0.0187102 ,
        0.01741016, 0.01664442, 0.01551878, 0.01464579, 0.01434892,
        0.01434264, 0.01427293, 0.01422656, 0.01314688, 0.01307341,
        0.01254079, 0.01206473, 0.01157919, 0.01072472, 0.01033127,
        0.01017585, 0.00984547, 0.00979725, 0.00968927, 0.00965488,
        0.00945497, 0.00925854, 0.00918946, 0.00914603, 0.00886098,
        0.0087972 , 0.00847617, 0.00752237, 0.00668958, 0.00594756,
        0.00593787, 0.00575206, 0.00563577, 0.00563094, 0.00548151,
        0.00535968, 0.00513506, 0.0050602 , 0.00505751, 0.00491947,
        0.00448701, 0.00433442, 0.00425157, 0.0042102 , 0.00385663,
        0.00347865, 0.00344685, 0.00341648, 0.00341472, 0.00332019,
        0.00331181, 0.00323448, 0.00323308, 0.00289994, 0.0028967 ,
        0.00273919, 0.00245938, 0.00245064, 0.0021973 , 0.00211695,
        0.00205618, 0.00205454, 0.00203374, 0.0020285 , 0.00195169,
        0.00191882, 0.00191739, 0.00189441, 0.00